In [1]:
import numpy as np
import time
from collections import namedtuple
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import qiskit as q
import qiskit.tools.jupyter
from qiskit.pulse import pulse_lib as _  # only want to call pulse_lib as q.pulse.pulse_lib

In [3]:
import pulse_compiler_helper_fns

In [4]:
q.IBMQ.load_account() # Load account from disk

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
provider = q.IBMQ.get_provider(group='qiskit-terra', project='pulse-testing')
backend = provider.get_backend('alt_almaden')
system = q.pulse.PulseChannelSpec.from_backend(backend)
cmd_def = q.pulse.CmdDef.from_defaults(backend.defaults().cmd_def, backend.defaults().pulse_library)
measure = cmd_def.get('measure', qubits=backend.configuration().meas_map[0])

/Users/alijavadi/Desktop/ajavadia-terra/qiskit/pulse/channels/pulse_channel_spec.py:87: DeprecationWarning: The PulseChannelSpec is deprecated. Use backend.configuration() instead. The supported methods require some migrations; check out the release notes for the complete details.
  DeprecationWarning)


In [6]:
thetas = np.linspace(0, np.pi/2, 20, endpoint=False)
print(thetas)

[0.         0.07853982 0.15707963 0.23561945 0.31415927 0.39269908
 0.4712389  0.54977871 0.62831853 0.70685835 0.78539816 0.86393798
 0.9424778  1.02101761 1.09955743 1.17809725 1.25663706 1.33517688
 1.41371669 1.49225651]


In [7]:
schedules = []
for theta in thetas:
    circuit = q.QuantumCircuit(2)
    circuit.h(0)
    circuit.h(1)
    circuit.zz_interaction(theta, 0, 1)
    circuit.h(0)
    circuit.h(1)

    decomposed_circuit = circuit.decompose()

    pulse_compiler_helper_fns.update_basis_gates_and_cmd_def(decomposed_circuit, backend, system, cmd_def)
    transpiled_circuit = q.transpile(decomposed_circuit, backend, optimization_level=1)

    schedule = q.schedule(transpiled_circuit, backend=backend, cmd_def=cmd_def)
    schedule |= measure << schedule.duration
    schedules.append(schedule)

In [8]:
shots = 1000
job = q.execute(schedules, backend=backend, shots=shots)
print('job is %s' % job.job_id())

job is 5dd702a6a903af00180af98d


/anaconda3/lib/python3.6/site-packages/ipywidgets/widgets/widget_layout.py:84: DeprecationWarning: Layout properties overflow_x and overflow_y have been deprecated and will be dropped in a future release. Please use the overflow shorthand property instead
  warnings.warn("Layout properties overflow_x and overflow_y have been deprecated and will be dropped in a future release. Please use the overflow shorthand property instead", DeprecationWarning)


In [10]:
job.status()

<JobStatus.DONE: 'job has successfully run'>

In [11]:
result = job.result()

In [12]:
from qiskit.ignis.verification.tomography.data import marginal_counts

In [13]:
counts_list = []
for i in range(len(thetas)):
    counts = marginal_counts(result.get_counts(i), meas_qubits=[0,1])    
    counts_list.append(counts)
    print(counts)

{'00': 958, '01': 30, '10': 11, '11': 1}
{'00': 942, '01': 33, '10': 16, '11': 9}
{'00': 955, '01': 24, '10': 4, '11': 17}
{'00': 911, '01': 45, '10': 14, '11': 30}
{'00': 884, '01': 29, '10': 16, '11': 71}
{'00': 861, '01': 30, '10': 19, '11': 90}
{'00': 800, '01': 27, '10': 36, '11': 137}
{'00': 786, '01': 38, '10': 27, '11': 149}
{'00': 770, '01': 43, '10': 21, '11': 166}
{'00': 692, '01': 42, '10': 31, '11': 235}
{'00': 665, '01': 48, '10': 35, '11': 252}
{'00': 628, '01': 50, '10': 44, '11': 278}
{'00': 636, '01': 40, '10': 38, '11': 286}
{'00': 605, '01': 51, '10': 47, '11': 297}
{'00': 598, '01': 55, '10': 37, '11': 310}
{'00': 520, '01': 58, '10': 60, '11': 362}
{'00': 498, '01': 60, '10': 62, '11': 380}
{'00': 516, '01': 55, '10': 38, '11': 391}
{'00': 517, '01': 64, '10': 53, '11': 366}
{'00': 514, '01': 57, '10': 44, '11': 385}


In [14]:
print(counts_list)

[{'00': 958, '01': 30, '10': 11, '11': 1}, {'00': 942, '01': 33, '10': 16, '11': 9}, {'00': 955, '01': 24, '10': 4, '11': 17}, {'00': 911, '01': 45, '10': 14, '11': 30}, {'00': 884, '01': 29, '10': 16, '11': 71}, {'00': 861, '01': 30, '10': 19, '11': 90}, {'00': 800, '01': 27, '10': 36, '11': 137}, {'00': 786, '01': 38, '10': 27, '11': 149}, {'00': 770, '01': 43, '10': 21, '11': 166}, {'00': 692, '01': 42, '10': 31, '11': 235}, {'00': 665, '01': 48, '10': 35, '11': 252}, {'00': 628, '01': 50, '10': 44, '11': 278}, {'00': 636, '01': 40, '10': 38, '11': 286}, {'00': 605, '01': 51, '10': 47, '11': 297}, {'00': 598, '01': 55, '10': 37, '11': 310}, {'00': 520, '01': 58, '10': 60, '11': 362}, {'00': 498, '01': 60, '10': 62, '11': 380}, {'00': 516, '01': 55, '10': 38, '11': 391}, {'00': 517, '01': 64, '10': 53, '11': 366}, {'00': 514, '01': 57, '10': 44, '11': 385}]
